# Create the metrical dictionarys
from words of the german wiktionary and the poetry corpus from projekt gutenberg

In [20]:
from transformers import AutoTokenizer
import pandas as pd
from rythm_utils import get_rythm, get_rythm_ipa, rythm_comp_adaptive
import math
from collections import Counter
import pickle
from tqdm.notebook import tqdm
import spacy
tqdm.pandas()
nlp = spacy.load("de_core_news_lg")

In [2]:
df = pd.read_csv('data_tools/wiktionary/wiktionary_data.csv')

In [3]:
df['rythm'] = df.progress_apply(lambda x: get_rythm_ipa(x.word,x.ipa),axis=1)

  0%|          | 0/758600 [00:00<?, ?it/s]

In [53]:
df.head(10)

,Unnamed: 0,word,ipa,label,rhyme,sep,rythm
0,0,hallo,['haˈloː'],substantiv,['oː'],"[['Hal·lo', 'Hal·los']]","[0.0, 1.0]"
1,1,subfamilia,[''],substantiv,[],"[['Sub·fa·mi·lia', 'Sub·fa·mi·li·ae']]",[0.5]
2,2,subregnum,['zʊpˈʁeːɡnʊm'],substantiv,[],"[['Sub·reg·num', 'Sub·reg·na']]","[0.0, 1.0, 0.0]"
3,3,subdivisio,['zʊpdiˈviːzi̯o'],substantiv,[],"[['Sub·di·vi·sio', 'Sub·di·vi·si·o·nes']]","[0.0, 0.0, 1.0, 0.0]"
4,4,phylum,['ˈfyːlʊm'],substantiv,['yːlʊm'],"[['Phy·lum', 'Phy·la']]","[1.0, 0.0]"
5,5,superphylum,['ˌzupɐˈfyːlʊm'],substantiv,['yːlʊm'],"[['Su·per·phy·lum', 'Su·per·phy·la']]","[0.5, 0.0, 1.0, 0.0]"
6,6,subphylum,['ˌzʊpˈfyːlʊm'],substantiv,['yːlʊm'],"[['Sub·phy·lum', 'Sub·phy·la']]","[0.5, 1.0, 0.0]"
7,7,subordo,['ˌzʊpˈʔɔʁdo'],substantiv,[],"[['Sub·or·do', 'Sub·or·di·nes']]","[0.5, 1.0, 0.0]"
8,8,subgenus,['ˌzʊpˈɡeːnʊs'],substantiv,['eːnʊs'],"[['Sub·ge·nus', 'Sub·ge·ne·ra']]","[0.5, 1.0, 0.0]"
9,9,subspezies,"['ˈzʊpʃpeːt͡si̯ɛs', 'ˈzʊpspeːt͡si̯ɛs']",substantiv,[],"[['Sub·spe·zi·es', 'Sub·spe·zi·es']]","[1.0, 0.0, 0.0, 0.0]"


In [5]:
stressed = ['substantiv',
 'verb',
 'adjektiv',
 'eigenname',
 'adverb',
 'gebundenes lexem',
 'deklinierte form',
 'vorname',
 'redewendung',
 'nachname',
 'konjugierte form',
 'kontraktion',
 'grußformel',
 'zahlklassifikator',
 'konjunktion',
 'numerale',
 'onomatopoetikum',
 'sprichwort',
 'pronomen',
 'partizip ii',
 'komparativ',
 'superlativ',
 'pronominaladverb',
 'geflügeltes wort',
 'modaladverb',
 'lokaladverb',
 'merkspruch',
 'partizip i',
 'formel',
 'postposition',
 'interrogativadverb',
 'wiederholungszahlwort',
 'erweiterter infinitiv',
 'adjektiv ',
 'pseudopartizip',
 'straßenname',
 'dekliniertes gerundivum']

In [6]:
rythm_df = df[['word','rythm']]
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if row['label'] in stressed and row['rythm'][0] == 0.5:
        rythm_df.at[index,'rythm'] = [1]
rythm_df.head()

  0%|          | 0/758600 [00:00<?, ?it/s]

,word,rythm
0,hallo,"[0.0, 1.0]"
1,subfamilia,[1]
2,subregnum,"[0.0, 1.0, 0.0]"
3,subdivisio,"[0.0, 0.0, 1.0, 0.0]"
4,phylum,"[1.0, 0.0]"


In [8]:
vocabulary = list(rythm_df['word'])

missing_vocab = []

gutenberg_vocab_df = pd.read_csv('data_tools/gutenberg/gutenberg_vocabulary.csv')
gutenberg_vocab = list(gutenberg_vocab_df['vocab'])

for word in tqdm(gutenberg_vocab): 
    if word not in vocabulary:
        missing_vocab.append(word)

  0%|          | 0/152904 [00:00<?, ?it/s]

In [13]:
gb_rythm_df = pd.DataFrame(missing_vocab, columns=['word'])

In [14]:
gb_rythm_df.head()

,word
0,görg
1,möcht
2,geschehn
3,heissem
4,nem


In [15]:
gb_rythm_df['rythm'] = gb_rythm_df.progress_apply(lambda x: get_rythm(x.word),axis=1)

  0%|          | 0/91499 [00:00<?, ?it/s]

In [18]:
gb_rythm_df.to_pickle('gutenberg_rythm.pik')

In [25]:
gb_rythm_df['label'] = gb_rythm_df.progress_apply(lambda x: nlp(x.word)[0].pos_,axis=1)

  0%|          | 0/91499 [00:00<?, ?it/s]

In [30]:
stressed = ['PROPN','ADV','AUX','ADJ','NOUN','VERB']
gb_rythm_df_proc = gb_rythm_df[['word','rythm']]
for index, row in tqdm(gb_rythm_df.iterrows(), total=gb_rythm_df.shape[0]):
    if row['label'] in stressed and row['rythm'][0] == 0.5:
        gb_rythm_df_proc.at[index,'rythm'] = [1]
gb_rythm_df_proc.head()

  0%|          | 0/91499 [00:00<?, ?it/s]

,word,rythm
0,görg,[1]
1,möcht,[1]
2,geschehn,"[0.0, 1.0]"
3,heissem,"[1.0, 0.0]"
4,nem,[0.5]


In [33]:
rythm_df_comp = pd.concat([rythm_df,gb_rythm_df_proc])

print(len(rythm_df))
print(len(rythm_df_comp))
rythm_df_comp.head()

758600
850099


,word,rythm
0,hallo,"[0.0, 1.0]"
1,subfamilia,[1]
2,subregnum,"[0.0, 1.0, 0.0]"
3,subdivisio,"[0.0, 0.0, 1.0, 0.0]"
4,phylum,"[1.0, 0.0]"


In [34]:
rythm_df_comp['matching'] = rythm_df_comp['rythm'].apply(lambda x: rythm_comp_adaptive(x,[[1,0],[0,1]],adaptive = True)
                                  if len(x) > 1  else True)
rythm_df_comp['start'] = rythm_df_comp['rythm'].apply(lambda x: max([math.ceil(x[0]) - x[1],0]) if len(x) > 1 else x[0]) # 0.5 -> 1 if x[1]=1 else 0.5 -> 0
rythm_df_comp['end'] = rythm_df_comp['rythm'].apply(lambda x: max([math.ceil(x[-1]) - x[-2],0]) if len(x) > 1 else x[0]) # 0.5 -> 1 if x[1]=1 else 0.5 -> 0
rythm_df_comp['num_syll'] = rythm_df_comp['rythm'].apply(lambda x: len(x)) 

In [35]:
rythm_df_comp.head()

,word,rythm,matching,start,end,num_syll
0,hallo,"[0.0, 1.0]",True,0.0,1.0,2
1,subfamilia,[1],True,1.0,1.0,1
2,subregnum,"[0.0, 1.0, 0.0]",True,0.0,0.0,3
3,subdivisio,"[0.0, 0.0, 1.0, 0.0]",False,0.0,0.0,4
4,phylum,"[1.0, 0.0]",True,1.0,0.0,2


In [36]:
df_matching = rythm_df_comp.drop(rythm_df_comp[rythm_df_comp.matching == False].index)

In [37]:
df_matching.head()

,word,rythm,matching,start,end,num_syll
0,hallo,"[0.0, 1.0]",True,0.0,1.0,2
1,subfamilia,[1],True,1.0,1.0,1
2,subregnum,"[0.0, 1.0, 0.0]",True,0.0,0.0,3
4,phylum,"[1.0, 0.0]",True,1.0,0.0,2
5,superphylum,[1],True,1.0,1.0,1


In [38]:
df_matching.to_csv('rythm_beam_search/word_rythm.csv')

In [40]:
# each word could possibly be preceded by a space and capital or not capital
tokenizer = AutoTokenizer.from_pretrained('Anjoe/german-poetry-gpt2-large')

df_matching['tokens'] = df_matching['word'].apply(lambda x: tokenizer(str(x))['input_ids'] )
df_matching['tokens_cap'] = df_matching['word'].apply(lambda x: tokenizer(str(x).capitalize())['input_ids'] )
df_matching['tokens_start'] = df_matching['word'].apply(lambda x: tokenizer(' '+ str(x))['input_ids'] )
df_matching['tokens_start_cap'] = df_matching['word'].apply(lambda x: tokenizer(' '+ str(x).capitalize())['input_ids'] )

/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [41]:
df_matching.head()

,word,rythm,matching,start,end,num_syll,tokens,tokens_cap,tokens_start,tokens_start_cap
0,hallo,"[0.0, 1.0]",True,0.0,1.0,2,[27137],[3694],[39424],[15350]
1,subfamilia,[1],True,1.0,1.0,1,"[31764, 70, 1115, 1101]","[38781, 70, 1115, 1101]","[13126, 70, 1115, 1101]","[10963, 70, 1115, 1101]"
2,subregnum,"[0.0, 1.0, 0.0]",True,0.0,0.0,3,"[31764, 2225, 78, 368]","[38781, 2225, 78, 368]","[13126, 2225, 78, 368]","[10963, 2225, 78, 368]"
4,phylum,"[1.0, 0.0]",True,1.0,0.0,2,"[1456, 6565, 368]","[4203, 6565, 368]","[5409, 6565, 368]","[1823, 6565, 368]"
5,superphylum,[1],True,1.0,1.0,1,"[31841, 1456, 6565, 368]","[9466, 1456, 6565, 368]","[2881, 1456, 6565, 368]","[4071, 1456, 6565, 368]"


In [43]:
df_matching.to_csv('rythm_beam_search/rythm_token.csv')

In [44]:
df_matching['length'] = df_matching.apply(lambda x: max(len(x.tokens), len(x.tokens_start)),axis = 1)

/home/andreas/.conda/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
df_matching.head()

,word,rythm,matching,start,end,num_syll,tokens,tokens_cap,tokens_start,tokens_start_cap,length
0,hallo,"[0.0, 1.0]",True,0.0,1.0,2,[27137],[3694],[39424],[15350],1
1,subfamilia,[1],True,1.0,1.0,1,"[31764, 70, 1115, 1101]","[38781, 70, 1115, 1101]","[13126, 70, 1115, 1101]","[10963, 70, 1115, 1101]",4
2,subregnum,"[0.0, 1.0, 0.0]",True,0.0,0.0,3,"[31764, 2225, 78, 368]","[38781, 2225, 78, 368]","[13126, 2225, 78, 368]","[10963, 2225, 78, 368]",4
4,phylum,"[1.0, 0.0]",True,1.0,0.0,2,"[1456, 6565, 368]","[4203, 6565, 368]","[5409, 6565, 368]","[1823, 6565, 368]",3
5,superphylum,[1],True,1.0,1.0,1,"[31841, 1456, 6565, 368]","[9466, 1456, 6565, 368]","[2881, 1456, 6565, 368]","[4071, 1456, 6565, 368]",4


In [47]:
df_matching_s = df_matching.drop(df_matching[df_matching.length > 6].index) # this filters a lot of not useful words

In [49]:
lst_0 = []
lst_1 = []

lst_0_s = []  #start of word
lst_1_s = []  #start of word
for i in range(1,6):
    for index, row in df_matching_s.iterrows():
        if row['length'] >= i and i > 1:
            if row['start'] == 0:
                lst_0 += [str(row['tokens'][:i]), str(row['tokens_start'][:i]),
                         str(row['tokens_cap'][:i]), str(row['tokens_start_cap'][:i])]
                
            else: 
                lst_1 += [str(row['tokens'][:i]), str(row['tokens_start'][:i]),
                         str(row['tokens_cap'][:i]), str(row['tokens_start_cap'][:i])]
                
        elif row['length'] >= i and i == 1:
            if row['start'] == 0:
                lst_0 += [str(row['tokens'][:i]),str(row['tokens_cap'][:i])]
                lst_0_s += [row['tokens_start'][0],row['tokens_start_cap'][0]]
                
            else: 
                lst_1 += [str(row['tokens'][:i]),str(row['tokens_cap'][:i])]
                lst_1_s += [row['tokens_start'][0],row['tokens_start_cap'][0]]

In [51]:
counts_0 = Counter(lst_0)
counts_1 = Counter(lst_1)
counts_0_s = Counter(lst_0_s)
counts_1_s = Counter(lst_1_s)

In [55]:
counts_1_s[13126]

112

In [56]:
with open('rythm_beam_search/notstressed', 'wb') as f:
    pickle.dump(counts_0,f)
    
with open('rythm_beam_search/stressed', 'wb') as f:
    pickle.dump(counts_1,f)
    
with open('rythm_beam_search/notstressed_start', 'wb') as f:
    pickle.dump(counts_0_s,f)
    
with open('rythm_beam_search/stressed_start', 'wb') as f:
    pickle.dump(counts_1_s,f)

In [7]:
# cell to extract the labels
labels = []
for index, row in df.iterrows():
    label = row['label']
    if label not in labels: 
        labels.append(label)

In [ ]:
# list of wiktionary labels
'''['substantiv',
 'personalpronomen',
 'verb',
 'adjektiv',
 'hilfsverb',
 'toponym',
 'eigenname',
 'abkürzung',
 'adverb',
 'gebundenes lexem',
 'deklinierte form',
 'partikel',
 'wortverbindung',
 'vorname',
 'redewendung',
 'indefinitpronomen',
 'präposition',
 'demonstrativpronomen',
 'interjektion',
 'temporaladverb',
 'nachname',
 'konjugierte form',
 'kontraktion',
 'grußformel',
 'subjunktion',
 'reflexivpronomen',
 'zahlklassifikator',
 'konjunktion',
 'numerale',
 'onomatopoetikum',
 'artikel',
 'sprichwort',
 'negationspartikel',
 'präfix',
 'pronomen',
 'antwortpartikel',
 nan,
 'gradpartikel',
 'partizip ii',
 'komparativ',
 'fokuspartikel',
 'superlativ',
 'konjunktionaladverb',
 'präfixoid',
 'relativpronomen',
 'interrogativpronomen',
 'pronominaladverb',
 'geflügeltes wort',
 'modaladverb',
 'possessivpronomen',
 'lokaladverb',
 'modalpartikel',
 'merkspruch',
 'partizip i',
 'abtönungspartikel',
 'formel',
 'postposition',
 'interrogativadverb',
 'wiederholungszahlwort',
 'reziprokpronomen',
 'erweiterter infinitiv',
 'adjektiv ',
 'pseudopartizip',
 'straßenname',
 'dekliniertes gerundivum',
 'enklitikon',
 'zirkumfix']'''